In [1]:
import shared.ml_config_core as ml_config_core
import pandas as pd
import seaborn as sns

from shared.ml_config_core import ModelConfigsCollection
# from src import feature_engineering, data_loader
from shared.ml_config_runner import run_tuning_for_configs_collection
from shared.definitions import TuningResult
from matplotlib import pyplot as plt
from shared import utils

from sklearn.model_selection import train_test_split

from Draft import feature_builder_v2
import importlib
import xgboost as xgb

import importlib

In [2]:
VERBOSE = True

In [3]:
importlib.reload(feature_builder_v2)
features_matrix = feature_builder_v2.load_datasets_and_prepare_features(drop_meta_data=True)


In [4]:
display("Distribution of Samples")
value_counts = features_matrix["target__TARGET"].value_counts()
df_value_counts = pd.DataFrame(
    {"Total Rows": value_counts, "Proportion": value_counts / len(features_matrix)}
)
df_value_counts.round(2)

'Distribution of Samples'

,Total Rows,Proportion
0,91990,0.92
1,8010,0.08


In [5]:
transformers = [
    # No feature engineering tuning for for rejected/accepted df
    # ml_config_core.AcceptedRejectedDropAllButEmployment,  # Only used for testing
]
configs: ModelConfigsCollection = {
    # "BaselineOnlyEmplDuration": ml_config_core.ModelPipelineConfig(
    #     model_config=ml_config_core.XGBoostTuneF1(),
    #     transformer_config=ml_config_core.PipelineTransformerConfig([ml_config_core.AcceptedRejectedDropAllButEmployment]),
    # ),
    # "XGBoostTunePRAUC": ml_config_core.ModelPipelineConfig(
    #     model_config=ml_config_core.XGBoostTunePRAUC(),
    #     transformer_config=ml_config_core.PipelineTransformerConfig(transformers),
    # ),
    # "XGBoostTuneF1Failed": ml_config_core.ModelPipelineConfig(
    #     model_config=ml_config_core.XGBoostTuneF1(),
    #     transformer_config=ml_config_core.PipelineTransformerConfig(transformers),
    # ),
    "LGBMTunePRAUC": ml_config_core.ModelPipelineConfig(
        model_config=ml_config_core.LGBMTuneF1(),
        transformer_config=ml_config_core.PipelineTransformerConfig(transformers),
    ),

}

In [6]:
tuning_results = run_tuning_for_configs_collection(configs, features_matrix)

[I 2024-04-08 16:09:51,806] A new study created in memory with name: no-name-39a309fa-7093-418e-a85f-d5b9a0546337
/home/paulius/data/projects/ppuodz-ML.4.1/shared/pipeline.py:629: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  params[key] = trial.suggest_int(key, value.start, value.end, value.step)
[W 2024-04-08 16:09:51,816] Trial 0 failed with parameters: {'class_weight': None, 'objective': 'binary', 'n_estimators': 525} because of the following error: TypeError('Trial.suggest_float() takes 4 positional arguments but 5 were given').
Traceback (most recent call last):
  File "/home/paulius/miniconda3/envs/rapids_v2/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/home/paulius/data/projects/ppuodz-ML.4.1/shared/pipeline.py", line 631, in objective
    params[key] = trial.suggest_float(key, value.start, value.end, value.step)
TypeError:

Tunning: - transformers: transformers: 0
 total options: 0
[]
search_grid:
{}

 - model: LGBMTunePRAUC n_iters=10 with:
 {'class_weight': ['balanced', None], 'objective': ['binary'], 'n_estimators': Range(100, 1000, 1, int), 'learning_rate': Range(0.01, 0.1, 1, float)}


TypeError: Trial.suggest_float() takes 4 positional arguments but 5 were given

In [9]:
TuningResult.convert_to_dataframe(tuning_results)

,best_score,best_params
model_key,,
LGBMTunePRAUC,0.542762,"{'model__subsample': 0.8, 'model__reg_lambda':..."


In [18]:
if VERBOSE:
    for model_key, model_data in tuning_results.items():
        transformer_tun_all_civ_results = model_data.transformer_tun_all_civ_results
        display(f"{model_key}:")
        display(
            transformer_tun_all_civ_results.sort_values(
                by=["mean_test_score"], ascending=False
            ).head(10)
        )

'XGBoostTunePRAUC:'

AttributeError: 'NoneType' object has no attribute 'sort_values'

In [27]:
if VERBOSE:
    for model_key, model_data in tuning_results.items():
        print(model_key)
        transformer_tun_all_civ_results = model_data.transformer_tun_all_civ_results
        display(transformer_tun_all_civ_results)

XGBoostTunePRAUC


KeyboardInterrupt: 